<a href="https://colab.research.google.com/github/elichen/aoc2024/blob/main/Day_17_Chronospatial_Computer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [368]:
input = """Register A: 63281501
Register B: 0
Register C: 0

Program: 2,4,1,5,7,5,4,5,0,3,1,6,5,5,3,0"""

In [369]:
def parse_input(input_str):
    lines = input_str.strip().split('\n')
    registers = {
        'A': int(lines[0].split(': ')[1]),
        'B': int(lines[1].split(': ')[1]),
        'C': int(lines[2].split(': ')[1])
    }
    program = [int(x) for x in lines[4].split(': ')[1].split(',')]
    return registers, program

def simulate_computer(registers, program, debug=False):
    r = registers.copy()  # Make a copy to avoid modifying the input
    ip = 0
    out = []

    def combo(operand):
        if operand < 4: return operand
        if operand == 4: return r['A']
        if operand == 5: return r['B']
        if operand == 6: return r['C']
        return None

    while ip < len(program):
        opcode = program[ip]
        operand = program[ip + 1]

        if opcode == 0:   # adv
            num = r['A']
            den = 2**combo(operand)
            r['A'] = num//den
            ip += 2
        elif opcode == 1: # bxl
            r['B'] = r['B'] ^ operand
            ip += 2
        elif opcode == 2: # bst
            r['B'] = combo(operand) % 8
            ip += 2
        elif opcode == 3: # jnz
            if r['A'] == 0:
              ip += 2
            else:
              ip = operand
        elif opcode == 4: # bxc
            r['B'] = r['B'] ^ r['C']
            ip += 2
        elif opcode == 5: # out
            out.append(combo(operand) % 8)
            if debug: print("out",combo(operand) % 8)
            ip += 2
        elif opcode == 6: # bdv
            num = r['A']
            den = 2**combo(operand)
            r['B'] = num//den
            ip += 2
        elif opcode == 7: # cdv
            num = r['A']
            den = 2**combo(operand)
            r['C'] = num//den
            ip += 2

        if debug: print(r,opcode,operand,ip)

    return r, out

# Example usage:
print(input)
registers, program = parse_input(input)
r, out = simulate_computer(registers, program)
print(",".join(map(str,out)))
print(r)

Register A: 63281501
Register B: 0
Register C: 0

Program: 2,4,1,5,7,5,4,5,0,3,1,6,5,5,3,0
3,4,3,1,7,6,5,6,0
{'A': 0, 'B': 0, 'C': 0}


In [370]:
def concatenate_bits(bit_list):
    result = 0
    for num in bit_list:
        # Shift the result left by 3 bits to make room for the next number
        result = (result << 3) | (num & 0b111)  # Ensure the number is treated as 3 bits
    return result

In [371]:
def try_sequence(sequence, target_output, program):
    registers = {'A': concatenate_bits(sequence), 'B': 0, 'C': 0}
    _, out = simulate_computer(registers, program)
    return out

def find_sequence(target_output, program, max_attempts=1000000):
    sequence = [1] + [0] * 15
    pos = 0  # Position in input sequence we're currently modifying
    attempts = 0

    while attempts < max_attempts:
        attempts += 1
        current_output = try_sequence(sequence, target_output, program)

        if attempts % 1000 == 0:
            print(f"\nAttempt {attempts}:")
            print(f"Current sequence: {sequence}")
            print(f"Current output:   {current_output}")
            print(f"Target output:    {target_output}")
            print(f"Current position: {pos}")

        if current_output == target_output:
            return sequence, attempts

        # Check if current input position gives correct output
        output_pos = 15 - pos
        if current_output[output_pos] == target_output[output_pos]:
            pos += 1  # Move to next input position
        else:
            sequence[pos] = (sequence[pos] + 1) % 8

            if sequence[pos] == 0:  # We've tried all values (0-7)
                # print(f"Tried all values at input[{pos}], backtracking...")
                # Backtrack to previous position
                while pos > 0:
                    pos -= 1
                    sequence[pos] = (sequence[pos] + 1) % 8
                    if sequence[pos] != 0:  # Found a position we can still increment
                        # print(f"Backtracked to position {pos}, new value: {sequence[pos]}")
                        # Reset all positions after this one
                        for i in range(pos + 1, 16):
                            sequence[i] = 0
                        break
                if pos == 0 and sequence[pos] == 0:
                    # print("Backtracked all the way to start, resetting...")
                    sequence = [1] + [0] * 15
                    pos = 0

    return None, attempts

sequence, attempts = find_sequence(program, program)
print(f"Found solution in {attempts} attempts:")
print(f"Input sequence: {sequence}")
print(f"Verification output: {try_sequence(sequence, program, program)}")
concatenate_bits(sequence)

Found solution in 163 attempts:
Input sequence: [3, 0, 6, 2, 3, 4, 5, 6, 1, 6, 1, 0, 4, 6, 3, 2]
Verification output: [2, 4, 1, 5, 7, 5, 4, 5, 0, 3, 1, 6, 5, 5, 3, 0]


109019930331546